In [ ]:
%load_ext autoreload
%autoreload 2

### Data Cleaning

In [ ]:
import pandas as pd


df = pd.read_csv("./data-june25-jul23.csv")

print(df.columns.tolist())

df.head()


In [ ]:
player_columns = ['Team 1 Player 1 ', 'Team 1 Player 2 ', 'Team 2 Player 1 ', 'Team 2 Player 2 ']
players = pd.unique(df[player_columns].values.ravel())

print(len(players))
players

In [ ]:
# Consolidate duplicate players (for now remove last name)
for col in player_columns:
    df[col] = df[col].str.split().str[0]

players = pd.unique(df[player_columns].values.ravel())

print(len(players))
players

In [ ]:
elo_scores: dict[str, float] = {player: 1000 for player in players}
elo_scores

### Basic ELO 
_Updating both players at the same time_

In [ ]:
elo_scores: dict[str, float] = {player: 1000 for player in players}

# Initialize ELO history with original scores
elo_history: dict[str, list[dict[str, float]]] = {}
elo_history["original"] = [dict(elo_scores)]  # Make original a list too for consistency

def expected_score_for_r1(R1, R2):
    exp = (R2 - R1)/400
    return 1/(1 + 10**exp)

# K set to 32 like chess
def calc_new_rating(old_rating, actual, expected):
    return old_rating + (32 * (actual - expected))

team1_columns = ['Team 1 Player 1 ', 'Team 1 Player 2 ']
team2_columns = ['Team 2 Player 1 ', 'Team 2 Player 2 ']

# Process each game
for row in df.to_dict(orient="records"):
    date = row["Date"]
    winning_team = row["Winning Team"]
    
    if winning_team == "Team 1":
        winning_players = [row[col] for col in team1_columns]
        losing_players = [row[col] for col in team2_columns]
    else:
        winning_players = [row[col] for col in team2_columns]
        losing_players = [row[col] for col in team1_columns]
    
    # Get averaged team elo
    winning_elo = (sum(elo_scores[player] for player in winning_players if player in elo_scores) / 2)
    losing_elo = (sum(elo_scores[player] for player in losing_players if player in elo_scores) / 2)
    
    # Calculate expected scores (total % adds to 1)
    winning_expected = expected_score_for_r1(winning_elo, losing_elo)
    losing_expected = 1 - winning_expected
    
    # Calculate new ratings (1 is a win, 0 is a loss)
    new_winning_elo = calc_new_rating(winning_elo, 1, winning_expected)
    new_losing_elo = calc_new_rating(losing_elo, 0, losing_expected)
    
    # Update ELO scores with rounding
    for player in winning_players:
        elo_scores[player] = round(new_winning_elo, 1)
    for player in losing_players:
        elo_scores[player] = round(new_losing_elo, 1)
    
    # Store current ELO state in history
    if date not in elo_history:
        elo_history[date] = []
    
    # Store a snapshot of current ELO scores after this game
    elo_history[date].append(dict(elo_scores))


# Calculate total games played
total_games = sum(len(scores_list) for date_key, scores_list in elo_history.items() if date_key != "original")

print(f"Total games played: {total_games}")
print("\nFinal ELO Rankings:")

# Sort players by final ELO score (highest to lowest)
sorted_players = sorted(elo_scores.items(), key=lambda x: x[1], reverse=True)

for rank, (player, elo) in enumerate(sorted_players, 1):
    print(f"{rank}. {player}: {elo}")


In [ ]:
import plotly.graph_objects as go

# Prepare data for plotting
x_positions = []
x_labels = []
all_player_data = {player: [] for player in players}

x_pos = 0

# Add original scores (first entry in the "original" list)
original_scores = elo_history["original"][0]
for player in players:
    all_player_data[player].append(original_scores[player])
x_positions.append(x_pos)
x_labels.append("Start")
x_pos += 1

# Process each date
for date_key in sorted(elo_history.keys()):
    if date_key == "original":
        continue
    
    games_on_date = elo_history[date_key]
    
    # Add each game for this date
    for game_scores in games_on_date:
        for player in players:
            all_player_data[player].append(game_scores[player])
        x_positions.append(x_pos)
        x_pos += 1
    
    # Add date label at the middle of games for this date
    if len(games_on_date) > 1:
        middle_pos = x_pos - len(games_on_date) + (len(games_on_date) - 1) / 2
    else:
        middle_pos = x_pos - 1
    x_labels.append((middle_pos, date_key))

# Create the figure
fig = go.Figure()

# Add a trace for each player
for player in players:
    fig.add_trace(go.Scatter(
        x=x_positions,
        y=all_player_data[player],
        mode='lines+markers',
        name=player,
        line=dict(width=2),
        marker=dict(size=6),
        hovertemplate=f'<b>{player}</b><br>ELO: %{{y}}<br>Game: %{{x}}<extra></extra>'
    ))

# Create custom x-axis labels
x_tick_positions = [0] + [pos for pos, label in x_labels[1:]]
x_tick_labels = ["Start"] + [label for pos, label in x_labels[1:]]

# Update layout
fig.update_layout(
    title={
        'text': 'ELO Rating Progression Over Time<br><sub>Click legend entries to show/hide players • Double-click to isolate</sub>',
        'x': 0.5,
        'font': {'size': 16}
    },
    xaxis_title='Date / Games',
    yaxis_title='ELO Rating',
    width=1000,
    height=600,
    hovermode='x unified',
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02
    ),
    margin=dict(r=150)  # Make room for legend
)

# Set custom x-axis ticks
fig.update_xaxes(
    tickmode='array',
    tickvals=x_tick_positions,
    ticktext=x_tick_labels,
    tickangle=45
)

# Add grid
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Show the plot
fig.show()


In [ ]:
# Export to html
fig.write_html("elo_rankings_jul23.html")